### Linear Regression Coefficient Significance Testing

In this notebook I will load the structured data and fit a statsmodels Linear Regression on it. I will investigate p-values, remove coefficients that are not significant, and retest. I will attempt to understand which, if any, are significant and why. 

In [3]:
import statsmodels.api as sm

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 500)

df = pd.read_pickle('./data/GOOG081320.pickle')
df.head()

,open,high,low,close,volume,date,SYMBOL,prev_close,diff_1,pct_change,log_return,pct_return_0,pct_return_1,pct_return_2,pct_return_3,pct_return_4,pct_return_5,pct_return_6,pct_return_7,pct_return_8,pct_return_9,pct_return_10,pct_return_11,pct_return_12,pct_return_13,pct_return_14,pct_return_15,pct_return_16,pct_return_17,pct_return_18,pct_return_19,pct_return_20,pct_return_21,pct_return_22,pct_return_23,pct_return_24,pct_return_25,pct_return_26,pct_return_27,pct_return_28,pct_return_29,pct_return_30,intervals,"monthly_(-0.12, -0.11499999999999999]","monthly_(-0.11499999999999999, -0.10999999999999999]","monthly_(-0.10999999999999999, -0.105]","monthly_(-0.105, -0.09999999999999999]","monthly_(-0.09999999999999999, -0.09499999999999999]","monthly_(-0.09499999999999999, -0.09]","monthly_(-0.09, -0.08499999999999999]","monthly_(-0.08499999999999999, -0.07999999999999999]","monthly_(-0.07999999999999999, -0.07499999999999998]","monthly_(-0.07499999999999998, -0.06999999999999998]","monthly_(-0.06999999999999998, -0.06499999999999999]","monthly_(-0.06499999999999999, -0.059999999999999984]","monthly_(-0.059999999999999984, -0.05499999999999998]","monthly_(-0.05499999999999998, -0.04999999999999999]","monthly_(-0.04999999999999999, -0.044999999999999984]","monthly_(-0.044999999999999984, -0.03999999999999998]","monthly_(-0.03999999999999998, -0.034999999999999976]","monthly_(-0.034999999999999976, -0.02999999999999997]","monthly_(-0.02999999999999997, -0.02499999999999998]","monthly_(-0.02499999999999998, -0.019999999999999976]","monthly_(-0.019999999999999976, -0.014999999999999972]","monthly_(-0.014999999999999972, -0.009999999999999981]","monthly_(-0.009999999999999981, -0.004999999999999977]","monthly_(-0.004999999999999977, 2.7755575615628914e-17]","monthly_(2.7755575615628914e-17, 0.005000000000000032]","monthly_(0.005000000000000032, 0.010000000000000037]","monthly_(0.010000000000000037, 0.015000000000000041]","monthly_(0.015000000000000041, 0.020000000000000018]","monthly_(0.020000000000000018, 0.025000000000000022]","monthly_(0.025000000000000022, 0.030000000000000027]","monthly_(0.030000000000000027, 0.03500000000000003]","monthly_(0.03500000000000003, 0.040000000000000036]","monthly_(0.040000000000000036, 0.04500000000000004]","monthly_(0.04500000000000004, 0.050000000000000044]","monthly_(0.050000000000000044, 0.05500000000000005]","monthly_(0.05500000000000005, 0.06000000000000005]","monthly_(0.06000000000000005, 0.06500000000000003]","monthly_(0.06500000000000003, 0.07000000000000003]","monthly_(0.07000000000000003, 0.07500000000000004]","monthly_(0.07500000000000004, 0.08000000000000004]","monthly_(0.08000000000000004, 0.08500000000000005]","monthly_(0.08500000000000005, 0.09000000000000005]","monthly_(0.09000000000000005, 0.09500000000000006]","monthly_(0.09500000000000006, 0.10000000000000003]","monthly_(0.10000000000000003, 0.10500000000000004]","monthly_(0.10500000000000004, 0.11000000000000004]","monthly_(0.11000000000000004, 0.11500000000000005]","monthly_(0.11500000000000005, 0.12000000000000005]","monthly_(0.12000000000000005, 0.12500000000000006]","monthly_(0.12500000000000006, 0.13000000000000006]","monthly_(0.13000000000000006, 0.13500000000000006]","monthly_(0.13500000000000006, 0.14000000000000007]","monthly_(0.14000000000000007, 0.14500000000000007]","monthly_(0.14500000000000007, 0.15000000000000008]","monthly_(0.15000000000000008, 0.15500000000000008]","monthly_(0.15500000000000008, 0.16000000000000003]","monthly_(0.16000000000000003, 0.16500000000000004]","monthly_(0.16500000000000004, 0.17000000000000004]","monthly_(0.17000000000000004, 0.17500000000000004]","monthly_(0.17500000000000004, 0.18000000000000005]","monthly_(0.18000000000000005, 0.18500000000000005]","monthly_(0.18500000000000005, 0.19000000000000006]","monthly_(0.19000000000000006, 0.19500000000000006]","monthly_(0.19500000000000006, 0.20000000000000007]","monthly_(0.20000000000000007, 0.20500000000000007]","monthly_(0.20500000000000007, 

In [2]:
df['target'] = df['pct_change'].shift(1)
df[['target','pct_change']]
drop_cols = ['pct_change',  # redundant column
             'intervals',  # data leakage
             'diff_1',  # dropping any column that references absolute price for now
             'prev_close', 
             'SYMBOL', # not important here
             'date', # should not be important 
             'open', # open high low close might be interesting but sticking with dropping absolute values 
             'high',
             'low',
             'close',
             'close_SMA_50', # since they are absolute still dropping these
             'close_SMA_120'] 

df.drop(drop_cols, axis=1, inplace=True)
df.dropna(inplace=True)

y = df['target']
X = df.drop('target', axis=1)

y.shape, X.shape

df.head()

,volume,log_return,pct_return_0,pct_return_1,pct_return_2,pct_return_3,pct_return_4,pct_return_5,pct_return_6,pct_return_7,pct_return_8,pct_return_9,pct_return_10,pct_return_11,pct_return_12,pct_return_13,pct_return_14,pct_return_15,pct_return_16,pct_return_17,pct_return_18,pct_return_19,pct_return_20,pct_return_21,pct_return_22,pct_return_23,pct_return_24,pct_return_25,pct_return_26,pct_return_27,pct_return_28,pct_return_29,pct_return_30,"monthly_(-0.12, -0.11499999999999999]","monthly_(-0.11499999999999999, -0.10999999999999999]","monthly_(-0.10999999999999999, -0.105]","monthly_(-0.105, -0.09999999999999999]","monthly_(-0.09999999999999999, -0.09499999999999999]","monthly_(-0.09499999999999999, -0.09]","monthly_(-0.09, -0.08499999999999999]","monthly_(-0.08499999999999999, -0.07999999999999999]","monthly_(-0.07999999999999999, -0.07499999999999998]","monthly_(-0.07499999999999998, -0.06999999999999998]","monthly_(-0.06999999999999998, -0.06499999999999999]","monthly_(-0.06499999999999999, -0.059999999999999984]","monthly_(-0.059999999999999984, -0.05499999999999998]","monthly_(-0.05499999999999998, -0.04999999999999999]","monthly_(-0.04999999999999999, -0.044999999999999984]","monthly_(-0.044999999999999984, -0.03999999999999998]","monthly_(-0.03999999999999998, -0.034999999999999976]","monthly_(-0.034999999999999976, -0.02999999999999997]","monthly_(-0.02999999999999997, -0.02499999999999998]","monthly_(-0.02499999999999998, -0.019999999999999976]","monthly_(-0.019999999999999976, -0.014999999999999972]","monthly_(-0.014999999999999972, -0.009999999999999981]","monthly_(-0.009999999999999981, -0.004999999999999977]","monthly_(-0.004999999999999977, 2.7755575615628914e-17]","monthly_(2.7755575615628914e-17, 0.005000000000000032]","monthly_(0.005000000000000032, 0.010000000000000037]","monthly_(0.010000000000000037, 0.015000000000000041]","monthly_(0.015000000000000041, 0.020000000000000018]","monthly_(0.020000000000000018, 0.025000000000000022]","monthly_(0.025000000000000022, 0.030000000000000027]","monthly_(0.030000000000000027, 0.03500000000000003]","monthly_(0.03500000000000003, 0.040000000000000036]","monthly_(0.040000000000000036, 0.04500000000000004]","monthly_(0.04500000000000004, 0.050000000000000044]","monthly_(0.050000000000000044, 0.05500000000000005]","monthly_(0.05500000000000005, 0.06000000000000005]","monthly_(0.06000000000000005, 0.06500000000000003]","monthly_(0.06500000000000003, 0.07000000000000003]","monthly_(0.07000000000000003, 0.07500000000000004]","monthly_(0.07500000000000004, 0.08000000000000004]","monthly_(0.08000000000000004, 0.08500000000000005]","monthly_(0.08500000000000005, 0.09000000000000005]","monthly_(0.09000000000000005, 0.09500000000000006]","monthly_(0.09500000000000006, 0.10000000000000003]","monthly_(0.10000000000000003, 0.10500000000000004]","monthly_(0.10500000000000004, 0.11000000000000004]","monthly_(0.11000000000000004, 0.11500000000000005]","monthly_(0.11500000000000005, 0.12000000000000005]","monthly_(0.12000000000000005, 0.12500000000000006]","monthly_(0.12500000000000006, 0.13000000000000006]","monthly_(0.13000000000000006, 0.13500000000000006]","monthly_(0.13500000000000006, 0.14000000000000007]","monthly_(0.14000000000000007, 0.14500000000000007]","monthly_(0.14500000000000007, 0.15000000000000008]","monthly_(0.15000000000000008, 0.15500000000000008]","monthly_(0.15500000000000008, 0.16000000000000003]","monthly_(0.16000000000000003, 0.16500000000000004]","monthly_(0.16500000000000004, 0.17000000000000004]","monthly_(0.17000000000000004, 0.17500000000000004]","monthly_(0.17500000000000004, 0.18000000000000005]","monthly_(0.18000000000000005, 0.18500000000000005]","monthly_(0.18500000000000005, 0.19000000000000006]","monthly_(0.19000000000000006, 0.19500000000000006]","monthly_(0.19500000000000006, 0.20000000000000007]","monthly_(0.20000000000000007, 0.20500000000000007]","monthly_(0.20500000000000007, 0.21000000000000008]","monthly_(0.21000000000000008, 0.2150000000000000

In [6]:
X.shape, y.shape

((3903, 171), (3903,))

In [7]:
X = sm.add_constant(X)
X

,const,volume,log_return,pct_return_0,pct_return_1,pct_return_2,pct_return_3,pct_return_4,pct_return_5,pct_return_6,pct_return_7,pct_return_8,pct_return_9,pct_return_10,pct_return_11,pct_return_12,pct_return_13,pct_return_14,pct_return_15,pct_return_16,pct_return_17,pct_return_18,pct_return_19,pct_return_20,pct_return_21,pct_return_22,pct_return_23,pct_return_24,pct_return_25,pct_return_26,pct_return_27,pct_return_28,pct_return_29,pct_return_30,"monthly_(-0.12, -0.11499999999999999]","monthly_(-0.11499999999999999, -0.10999999999999999]","monthly_(-0.10999999999999999, -0.105]","monthly_(-0.105, -0.09999999999999999]","monthly_(-0.09999999999999999, -0.09499999999999999]","monthly_(-0.09499999999999999, -0.09]","monthly_(-0.09, -0.08499999999999999]","monthly_(-0.08499999999999999, -0.07999999999999999]","monthly_(-0.07999999999999999, -0.07499999999999998]","monthly_(-0.07499999999999998, -0.06999999999999998]","monthly_(-0.06999999999999998, -0.06499999999999999]","monthly_(-0.06499999999999999, -0.059999999999999984]","monthly_(-0.059999999999999984, -0.05499999999999998]","monthly_(-0.05499999999999998, -0.04999999999999999]","monthly_(-0.04999999999999999, -0.044999999999999984]","monthly_(-0.044999999999999984, -0.03999999999999998]","monthly_(-0.03999999999999998, -0.034999999999999976]","monthly_(-0.034999999999999976, -0.02999999999999997]","monthly_(-0.02999999999999997, -0.02499999999999998]","monthly_(-0.02499999999999998, -0.019999999999999976]","monthly_(-0.019999999999999976, -0.014999999999999972]","monthly_(-0.014999999999999972, -0.009999999999999981]","monthly_(-0.009999999999999981, -0.004999999999999977]","monthly_(-0.004999999999999977, 2.7755575615628914e-17]","monthly_(2.7755575615628914e-17, 0.005000000000000032]","monthly_(0.005000000000000032, 0.010000000000000037]","monthly_(0.010000000000000037, 0.015000000000000041]","monthly_(0.015000000000000041, 0.020000000000000018]","monthly_(0.020000000000000018, 0.025000000000000022]","monthly_(0.025000000000000022, 0.030000000000000027]","monthly_(0.030000000000000027, 0.03500000000000003]","monthly_(0.03500000000000003, 0.040000000000000036]","monthly_(0.040000000000000036, 0.04500000000000004]","monthly_(0.04500000000000004, 0.050000000000000044]","monthly_(0.050000000000000044, 0.05500000000000005]","monthly_(0.05500000000000005, 0.06000000000000005]","monthly_(0.06000000000000005, 0.06500000000000003]","monthly_(0.06500000000000003, 0.07000000000000003]","monthly_(0.07000000000000003, 0.07500000000000004]","monthly_(0.07500000000000004, 0.08000000000000004]","monthly_(0.08000000000000004, 0.08500000000000005]","monthly_(0.08500000000000005, 0.09000000000000005]","monthly_(0.09000000000000005, 0.09500000000000006]","monthly_(0.09500000000000006, 0.10000000000000003]","monthly_(0.10000000000000003, 0.10500000000000004]","monthly_(0.10500000000000004, 0.11000000000000004]","monthly_(0.11000000000000004, 0.11500000000000005]","monthly_(0.11500000000000005, 0.12000000000000005]","monthly_(0.12000000000000005, 0.12500000000000006]","monthly_(0.12500000000000006, 0.13000000000000006]","monthly_(0.13000000000000006, 0.13500000000000006]","monthly_(0.13500000000000006, 0.14000000000000007]","monthly_(0.14000000000000007, 0.14500000000000007]","monthly_(0.14500000000000007, 0.15000000000000008]","monthly_(0.15000000000000008, 0.15500000000000008]","monthly_(0.15500000000000008, 0.16000000000000003]","monthly_(0.16000000000000003, 0.16500000000000004]","monthly_(0.16500000000000004, 0.17000000000000004]","monthly_(0.17000000000000004, 0.17500000000000004]","monthly_(0.17500000000000004, 0.18000000000000005]","monthly_(0.18000000000000005, 0.18500000000000005]","monthly_(0.18500000000000005, 0.19000000000000006]","monthly_(0.19000000000000006, 0.19500000000000006]","monthly_(0.19500000000000006, 0.20000000000000007]","monthly_(0.20000000000000007, 0.20500000000000007]","monthly_(0.20500000000000007, 0.21000000000000008]","monthly_(0.21000000000000008, 0.2150000000

In [8]:
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 target   R-squared:                       0.074
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     2.178
Date:                Fri, 14 Aug 2020   Prob (F-statistic):           1.94e-13
Time:                        17:25:24   Log-Likelihood:                 10143.
No. Observations:                3903   AIC:                        -2.001e+04
Df Residuals:                    3763   BIC:                        -1.913e+04
Df Model:                         139                                         
Covariance Type:            nonrobust                                         
                                                              coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------

/opt/conda/lib/python3.7/site-packages/statsmodels/base/model.py:1362: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/opt/conda/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/opt/conda/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/opt/conda/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [11]:
p_coefs = results.pvalues > 0.05

In [14]:
X.columns[p_coefs].shape

(106,)

In [15]:
X[X.columns[p_coefs]]

,const,volume,log_return,pct_return_0,pct_return_26,pct_return_27,pct_return_30,"monthly_(-0.12, -0.11499999999999999]","monthly_(-0.11499999999999999, -0.10999999999999999]","monthly_(-0.10999999999999999, -0.105]","monthly_(-0.105, -0.09999999999999999]","monthly_(-0.09999999999999999, -0.09499999999999999]","monthly_(-0.09499999999999999, -0.09]","monthly_(-0.09, -0.08499999999999999]","monthly_(-0.06499999999999999, -0.059999999999999984]","monthly_(-0.059999999999999984, -0.05499999999999998]","monthly_(-0.05499999999999998, -0.04999999999999999]","monthly_(-0.04999999999999999, -0.044999999999999984]","monthly_(-0.044999999999999984, -0.03999999999999998]","monthly_(-0.034999999999999976, -0.02999999999999997]","monthly_(-0.02999999999999997, -0.02499999999999998]","monthly_(-0.02499999999999998, -0.019999999999999976]","monthly_(-0.009999999999999981, -0.004999999999999977]","monthly_(0.010000000000000037, 0.015000000000000041]","monthly_(0.015000000000000041, 0.020000000000000018]","monthly_(0.020000000000000018, 0.025000000000000022]","monthly_(0.025000000000000022, 0.030000000000000027]","monthly_(0.03500000000000003, 0.040000000000000036]","monthly_(0.040000000000000036, 0.04500000000000004]","monthly_(0.04500000000000004, 0.050000000000000044]","monthly_(0.050000000000000044, 0.05500000000000005]","monthly_(0.05500000000000005, 0.06000000000000005]","monthly_(0.06000000000000005, 0.06500000000000003]","monthly_(0.06500000000000003, 0.07000000000000003]","monthly_(0.07500000000000004, 0.08000000000000004]","monthly_(0.08000000000000004, 0.08500000000000005]","monthly_(0.08500000000000005, 0.09000000000000005]","monthly_(0.09000000000000005, 0.09500000000000006]","monthly_(0.09500000000000006, 0.10000000000000003]","monthly_(0.10000000000000003, 0.10500000000000004]","monthly_(0.10500000000000004, 0.11000000000000004]","monthly_(0.11000000000000004, 0.11500000000000005]","monthly_(0.11500000000000005, 0.12000000000000005]","monthly_(0.12000000000000005, 0.12500000000000006]","monthly_(0.12500000000000006, 0.13000000000000006]","monthly_(0.13000000000000006, 0.13500000000000006]","monthly_(0.13500000000000006, 0.14000000000000007]","monthly_(0.14000000000000007, 0.14500000000000007]","monthly_(0.14500000000000007, 0.15000000000000008]","monthly_(0.15000000000000008, 0.15500000000000008]","monthly_(0.15500000000000008, 0.16000000000000003]","monthly_(0.16000000000000003, 0.16500000000000004]","monthly_(0.16500000000000004, 0.17000000000000004]","monthly_(0.17000000000000004, 0.17500000000000004]","monthly_(0.17500000000000004, 0.18000000000000005]","monthly_(0.18000000000000005, 0.18500000000000005]","monthly_(0.18500000000000005, 0.19000000000000006]","monthly_(0.19000000000000006, 0.19500000000000006]","monthly_(0.19500000000000006, 0.20000000000000007]","monthly_(0.20000000000000007, 0.20500000000000007]","monthly_(0.20500000000000007, 0.21000000000000008]","monthly_(0.21000000000000008, 0.21500000000000005]","yearly_(-0.12, -0.11499999999999999]","yearly_(-0.11499999999999999, -0.10999999999999999]","yearly_(-0.10999999999999999, -0.105]","yearly_(-0.105, -0.09999999999999999]","yearly_(-0.09999999999999999, -0.09499999999999999]","yearly_(-0.09499999999999999, -0.09]","yearly_(-0.09, -0.08499999999999999]","yearly_(-0.08499999999999999, -0.07999999999999999]","yearly_(-0.07999999999999999, -0.07499999999999998]","yearly_(-0.07499999999999998, -0.06999999999999998]","yearly_(-0.06999999999999998, -0.06499999999999999]","yearly_(-0.06499999999999999, -0.059999999999999984]","yearly_(-0.059999999999999984, -0.05499999999999998]","yearly_(-0.05499999999999998, -0.04999999999999999]","yearly_(-0.04999999999999999, -0.044999999999999984]","yearly_(-0.044999999999999984, -0.03999999999999998]","yearly_(-0.034999999999999976, -0.02999999999999997]","yearly_(-0.02999999999999997, -0.02499999999999998]","yearly_(-0.02499999999999998, -0.019999999999999976]","yearly_(-0.019999999999999976, -0.014999999999999972]","yearly_(-0

In [16]:
mod2 = sm.OLS(y, X[X.columns[p_coefs]])
res2 = mod2.fit()
print(res2.summary())

                            OLS Regression Results                            
Dep. Variable:                 target   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     1.594
Date:                Fri, 14 Aug 2020   Prob (F-statistic):           0.000439
Time:                        17:30:13   Log-Likelihood:                 10062.
No. Observations:                3903   AIC:                        -1.995e+04
Df Residuals:                    3815   BIC:                        -1.940e+04
Df Model:                          87                                         
Covariance Type:            nonrobust                                         
                                                             coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------

Testing for significance seems to have null or negative effect on the result. 

Many of the distribution features created so far are surprisingly correlated to the target value, with coefficient pvalues far exceeding 0.05. However, despite their significance in this case we still are not able to predict much information more than the mean. 